# Projeto 1 - Ciência dos Dados

Nome: Denis Alonso

Nome: Felipe Menke

Nome: Gabriel Correia

___
### Atenção: <br> $\quad$* Serão permitidos grupos de até 3 pessoas, mas com uma rubrica mais exigente. <br> $\quad$* Veja RUBRICA na última página do arquivo PDF que traz o enunciado do Proejto 1.

___
Carregando algumas bibliotecas:

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\lipem\OneDrive\Documentos\GitHub\PyGAME\Projeto1-Cdados-Insper


___

### Carregando a base de dados com as mensagens dos seus arquivos. <br> Tire o `#` do início da linha de código condizente ao caso escolhido para o projeto 1.

In [3]:
# SE SEU PROJETO USA OS DADOS SOBRE "Airline Passenger Reviews"
train = pd.read_csv('dados_treino_ate_TRIO_FelipeMenke.csv')
test = pd.read_csv('dados_teste_ate_TRIO_FelipeMenke.csv')

In [4]:
# SE SEU PROJETO USA OS DADOS SOBRE "ChatGPT Classification"
#train = pd.read_csv('dados_treino_so_DUPLA_'+nome+'.csv')
#test = pd.read_csv('dados_teste_so_DUPLA_'+nome+'.csv')

In [5]:
train.head()

,Review,Target
0,Singapore to Jakarta. 9 years since I last to...,Passive
1,Norwegian Long Haul. OSL-LGW. Flight DY1310. 1...,Promoter
2,Guangzhou to Paris. I have paid for inflight...,Promoter
3,Vientiane to Melbourne via Bangkok. A very en...,Promoter
4,I travelled Manchester - Sydney return on 777-...,Passive


In [6]:
test.head()

,Review,Target
0,Budapest to Warsaw. The ground crew was very ...,Passive
1,Bergerac to Stansted. Just come off a particul...,Detractor
2,BRU-LIS. Flight left a few minutes late aircra...,Promoter
3,Fleet of A319 and A320 in this route. Fast che...,Passive
4,4 Nov Houston-Doha QR714 and 15 Nov Doha-Houst...,Passive


___
# Classificador Automático (Boot)


Faça aqui uma descrição do seu assunto e o contexto referente aos rótulos cujas mensagens (ou reviews) deverão ser classificadas.

ESCREVA AQUI...

___
## Montando SEU Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [7]:
#Função para limpar o texto


import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$'']' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

**Todas as Avaliações**

In [8]:
#Descobrindo a quantidade total de palavras
dic_total = train.loc[:, "Review"]  
palavras_totais = dic_total.tolist()  
palavras_totais = " ".join(palavras_totais)  
palavras_totais = cleanup(palavras_totais).lower()  
palavras_totais = palavras_totais.split()  
palavras_totais = pd.Series(palavras_totais)  
qtd_palavras_totais = palavras_totais.value_counts()  
qtd_palavras_totais = qtd_palavras_totais.sum() 


**Dados - Avaliações Perfil Passivo**


In [ ]:
#PASSIVO

dic_passivos = train.loc[train["Target"]=="Passive","Review"]
#Transformando em Lista
frases_passivos = dic_passivos.tolist()

#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_passivos = " ".join(frases_passivos)
frases_passivos = cleanup(frases_passivos).lower()
frases_passivos = frases_passivos.split()

#Probabilidade de ser Passivo
qtd_palavras_passivos = pd.Series(frases_passivos).value_counts()
qtd_palavras_passivos = qtd_palavras_passivos.sum()

Prob_Passivo = qtd_palavras_passivos/qtd_palavras_totais

#Criando Frequencia Relativa das Palavras

freq_rel_passivos = pd.Series(frases_passivos).value_counts(True)

**Dados - Avaliações Perfil Detrator**

In [10]:
#DETRACTOR

dic_detrator = train.loc[train["Target"]=="Detractor","Review"]

#Transformando em Lista
frases_detractor = dic_detrator.tolist()
#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_detractor = " ".join(frases_detractor)
frases_detractor = cleanup(frases_detractor).lower()
frases_detractor = frases_detractor.split()

#Probabilidade de ser Detrator
qtd_palavras_detrator = pd.Series(frases_detractor).value_counts()
qtd_palavras_detrator = qtd_palavras_detrator.sum()

Prob_Detrator = qtd_palavras_detrator/qtd_palavras_totais

#Criando Frequencia Relativa das Palavras

freq_rel_detrator = pd.Series(frases_detractor).value_counts(True)

**Dados - Avaliações Perfil Promoter**

In [11]:
#PROMOTER

dic_promoter = train.loc[train["Target"]=="Promoter","Review"]

#Transformando em Lista
frases_promoter = dic_promoter.tolist()

#Juntando as frases,Limpando (caracteres especiais, e diminuindo as letras)
frases_promoter = " ".join(frases_promoter)
frases_promoter = cleanup(frases_promoter).lower()
frases_promoter = frases_promoter.split()

#Probabilidade de ser Detrator
qtd_palavras_promoter = pd.Series(frases_promoter).value_counts()
qtd_palavras_promoter = qtd_palavras_promoter.sum()

Prob_Promoter = qtd_palavras_promoter/qtd_palavras_totais

#Criando Frequencia Relativa das Palavras

freq_rel_promoter = pd.Series(frases_promoter).value_counts(True)
print(freq_rel_promoter)

the         0.049208
and         0.037813
to          0.029852
was         0.026083
a           0.021977
              ...   
737800ng    0.000011
sep/13      0.000011
eilna       0.000011
787800      0.000011
dy1310      0.000011
Name: proportion, Length: 6755, dtype: float64


___
## Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [24]:
dic_final = {}
Target_test = test.loc[:, "Target"]
Review_test = test.loc[:, "Review"]

for i in range(len(test)):
    tipo = Target_test.iloc[i]  
    frase = cleanup(Review_test.iloc[i]).lower().strip()
    frase = " ".join(frase.split())  

    
    probDetratordadoFrase = 1
    probPromoterdadoFrase = 1
    probPassivodadoFrase = 1


    for palavra in frase.split():
        palavra = palavra.lower().strip()


        probDetratordadoFrase *= freq_rel_detrator.get(palavra, 1)
        probPromoterdadoFrase *= freq_rel_promoter.get(palavra, 1)
        probPassivodadoFrase *= freq_rel_passivos.get(palavra, 1)

    maior = max(probDetratordadoFrase, probPromoterdadoFrase, probPassivodadoFrase)

    if maior == probDetratordadoFrase:
        final = "Detractor"
    elif maior == probPromoterdadoFrase:
        final = "Promoter"
    else:
        final = "Passive"

    dic_final[i] = (tipo, final)


import pandas as pd
df_resultado = pd.DataFrame.from_dict(dic_final, orient="index", columns=["Target", "Predição"])


df_resultado

,Target,Predição
0,Passive,Passive
1,Detractor,Detractor
2,Promoter,Promoter
3,Passive,Passive
4,Passive,Detractor
...,...,...
1075,Passive,Promoter
1076,Passive,Detractor
1077,Detractor,Promoter
1078,Detractor,Detractor


In [ ]:
# Selecionando apenas as categorias corretas
Passivo = df_resultado.loc[df_resultado["Target"] == "Passive", "Predição"]
Promoter = df_resultado.loc[df_resultado["Target"] == "Promoter", "Predição"]
Detractor = df_resultado.loc[df_resultado["Target"] == "Detractor", "Predição"]

# Inicializando contadores
acerto_passivo = 0
erro_passivo = 0
acerto_promoter = 0
erro_promoter = 0
acerto_detrator = 0
erro_detrator = 0

# Loop corrigido - agora usa `.iteritems()` para evitar problemas de índice
for idx, predicao in Passivo.items():
    if predicao == "Passive":
        acerto_passivo += 1
    else:
        erro_passivo += 1

for idx, predicao in Promoter.items():
    if predicao == "Promoter":
        acerto_promoter += 1
    else:
        erro_promoter += 1

for idx, predicao in Detractor.items():
    if predicao == "Detractor":
        acerto_detrator += 1
    else:
        erro_detrator += 1

# Exibir os resultados de acertos e erros
print(f"Passivo: {acerto_passivo} acertos, {erro_passivo} erros")
print(f"Promoter: {acerto_promoter} acertos, {erro_promoter} erros")
print(f"Detractor: {acerto_detrator} acertos, {erro_detrator} erros")


Passivo: 67 acertos, 305 erros
Promoter: 149 acertos, 192 erros
Detractor: 243 acertos, 124 erros


___
## Análise Qualitativa da Performance do Classificador

**Criando a Acurácia dos Preditores**

___
## Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Leia atentamente a rubrica colocada no enunciado do Projeto 1 (última página). <br>
Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens, mas tendem a melhorar na classificação das mensagens. Ex: stemming, lemmatization, stopwords.
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar novas mensagens classificadas pelo próprio classificador como amostra de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* REFLETE criticamente sobre os resultados obtidos, identificando limitações do modelo e sugerindo possíveis melhorias ou diferentes abordagens com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa).
* DOCUMENTOU bem o código, com explicações claras para cada etapa do processo, incluindo o raciocínio por trás das decisões de modelagem e das transformações de dados.


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[Natural Language Processing (Part 17)-Laplacian Smoothing](https://medium.com/@Coursesteach/natural-language-processing-part-17-laplacian-smoothing-7d4be71d0ded) **Mais simples**